In [ ]:
txtpath='XXXXXX'
text=open(txtpath,'r')

In [ ]:
# 去掉文本中的空格
text_in_lines=text.split('\n')
text_in_lines=[lines for lines in text_in_lines if len(lines)>0]

In [ ]:
# 建立汉字和数字的映射关系
def create_lookup_tables(input_data):
    vocab=set(input_data)
    
    vocab2int = {word: idx for idx,word in enumerate(vocab)}
    int2vocab = dict(enumerate(vocab))
    
    return vocab2int,int2vocab

# 创建一个符号查询表，把标点符号与一组标志一一对应，排除标点符号的影响
symbols = set(['。', '，', '“', "”", '；', '！', '？', '（', '）', '——', '\n'])    
tokens = ["P", "C", "Q", "T", "S", "E", "M", "I", "O", "D", "R"]
token_dict=dict(zip(symbols, tokens))

In [ ]:
# 处理文本
import pickle

for key, token in token_dict.items():
    text = text.replace(key,'{}'.format(token))

text = list(text)

vocab2int,int2vocab = create_lookup_tables(text)
int_text = [vocab2int[word] for word in text]
pickle.dump((int_text, vocab2int, int2vocab, token_dict), open('preprocess.p', 'wb'))


int_text, vocab_to_int, int_to_vocab, token_dict = pickle.load(open('preprocess.p', mode='rb'))

到这里就处理完原始文本数据了，接下来搭建模型，炼丹。

In [ ]:
#超参数的设置
num_epochs = 200

batch_size = 256

rnn_size = 512

embed_dim = 512

seq_length = 30

l_r=0.003

inf_show_batchs = 30

save_dir='./save'


In [3]:
def get_inputs():
    inputs = tf.placeholder(tf.int32, [None,None], name='inputs')
    targets = tf.placeholder(tf.int32, [None,None], name='targets')
    l_r=tf.placeholder(tf.float32,name='learning_rate')
    
    return inputs,targets,l_r

def get_init_cell(batch_size, rnn_size):
    num_layers = 2
    keep_prob = 0.8
    
    #创建包含rnn_size的神经元的lstm cell
    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
    #使用dropout防止过拟合
    drop=tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob =keep_prob)
    
    #创建2层lstm层
    cell=tf.contrib.rnn.MultiRNNCell([drop for _ in range(num_layers)])
    
    #初始化状态为0.0
    init_state = cell.zero_state(batch_size, tf.float32)
    
    #后面生成文字的时候使用这个名字找到缓存的state
    init_state = tf.identity(init_state, name='init_state')
    
    return cell,init_state


SyntaxError: unexpected EOF while parsing (<ipython-input-3-9e41aa7538c8>, line 8)

In [ ]:
def build_rnn(cell, inputs):
    outputs,final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    
    #同样给final_state一个名字，后面要重新获取缓存
    final_state = tf.identify(final_state, name='final_state')
    
    return outputs,final_state


def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    
    #创建embedding layer
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim)), dtype=tf.float32)
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    #计算outputs和final_state
    outputs, final_state =build_rnn(cell, embed)
    
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None,
                                              weights_initializer = tf.truncated_normal_initializer(stddev=0.1),
                                              biases_initializer = tf.zeros_initializer())
    return logits, final_state

In [ ]:
#搭建整个RNN网络模型
from tensorflow.contrib import seq2seq

with tf.Graph().as_default():
    #文字总量
    vocab_size = len(int2vocab)
    
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    
    cell, inital_state = get_init_cell(input_data_shape[0],rnn_size)
    
    logits, final_state =build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)
    
    probs=tf.nn.softmax(logits,name='probs')
    
    #计算loss
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))
    
    optimizer = tf.train.AdamOptimizer(lr)
    
    #将最后的梯度控制在[-1,1]范围内
    gradients =optimizer.compute_gradients(cost)
    capped_gradients =[(tf.clip_by_value(grad, -1., 1.), var) for grad,var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)
    

In [4]:
#训练模型

#使用get_batches方法一次使用一部分数据来训练
def get_batches(int_text, batch_size, seq_length):
    
    # 计算有多少个batch可以创建
    n_batches = (len(int_text) // (batch_size * seq_length))

    # 计算每一步的原始数据，和位移一位之后的数据
    batch_origin = np.array(int_text[: n_batches * batch_size * seq_length])
    batch_shifted = np.array(int_text[1: n_batches * batch_size * seq_length + 1])
    
    # 将位移之后的数据的最后一位，设置成原始数据的第一位，相当于在做循环
    batch_shifted[-1] = batch_origin[0]
    
    batch_origin_reshape = np.split(batch_origin.reshape(batch_size, -1), n_batches, 1)
    batch_shifted_reshape = np.split(batch_shifted.reshape(batch_size, -1), n_batches, 1)

    batches = np.array(list(zip(batch_origin_reshape, batch_shifted_reshape)))
    
    return batches
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variable_initializer())
    
    for epoch_i in range(num_epochs):
        state =sess.run(initial_state, {input_text:batches[0][0]})
        
        for batch_i,(x,y) in enumerate(batches):
            feed = {
                input_text:x,
                targets:y,
                initial_state:state,
                lr:l_r}
            train_loss, state, _ = sess.run([cost, final_state,train_op],feed)
            
            #打印训练信息
            if (epoch_i * len(batches) + batch_i)%inf_show_batchs ==0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))        
    
    #保存模型
    saver = tf.train.Saver()
    saver.save(sess,save_dir)
    print('Model Trained and Saved!!!!')

SyntaxError: unexpected EOF while parsing (<ipython-input-4-15677bc7bc40>, line 24)

In [ ]:
#保存使用到的变量，以便下次直接读取
pickle.dump((seq_length, save_dir), open('params.p', 'wb'))


开始使用训练好的模型生成文本

In [5]:
_, vocab_to_int, int_to_vocab, token_dict = pickle.load(open('preprocess.p', mode='rb'))
seq_length, load_dir=pickle.load(open('params.p', mode='rb'))


def get_tensors(loaded_graph):
   
    inputs = loaded_graph.get_tensor_by_name("inputs:0")
    
    initial_state = loaded_graph.get_tensor_by_name("init_state:0")
    
    final_state = loaded_graph.get_tensor_by_name("final_state:0")
    
    probs = loaded_graph.get_tensor_by_name("probs:0")
    
    return inputs, initial_state, final_state, probs


def pick_word(probabilities, int_to_vocab):
   
    chances = []
    
    for idx, prob in enumerate(probabilities):
        if prob >= 0.05:
            chances.append(int_to_vocab[idx])
    
    rand = np.random.randint(0, len(chances))
    
    return str(chances[rand])

# 生成文本的长度
gen_length = 500

# 文章开头的字，指定一个即可，这个字必须是在训练词汇列表中的
prime_word = '章'


loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # 加载保存过的session
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # 通过名称获取缓存的tensor
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # 准备开始生成文本
    gen_sentences = [prime_word]
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # 开始生成文本
    for n in range(gen_length):
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length - 1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # 将标点符号还原
    novel = ''.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '（', '“'] else ''
        novel = novel.replace(token.lower(), key)
    novel = novel.replace('\n ', '\n')
    novel = novel.replace('（ ', '（')
        
    print(novel)

SyntaxError: invalid syntax (<ipython-input-5-62ebafe5f044>, line 2)